This notebook serves the dual purpose of being the official submission & a tutorial for NLI (Natural Language Inference). Natural Language Inferencing (NLI) is a popular NLP problem that involves determining how pairs of sentences (consisting of a premise and a hypothesis) are related.

In this notebook, we shall have a go at the [Contradictory, My Dear Watson](https://www.kaggle.com/c/contradictory-my-dear-watson/overview/description) competition dataset by performing a simple EDA & then build/train a model with a pre-trained BERT Model for encoding different languages. The whole thing will be implemented using Tensorflow / Keras.

The training set contains a premise, a hypothesis, a label (0 = entailment, 1 = neutral, 2 = contradiction), and the language of the text.

I shall keep the notebook well commented & organized for easy reading :-).

# Libraries

In [ ]:
# Generic
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, warnings,json
warnings.filterwarnings('ignore')

# Plot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Transformer Model
from transformers import XLMRobertaTokenizer, TFXLMRobertaModel

# SKLearn Library
from sklearn.model_selection import train_test_split

# Garbage Collector
import gc

os.environ["WANDB_API_KEY"] = "0"

In [ ]:
# Initialize TPU

def Init_TPU():  

    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        REPLICAS = strategy.num_replicas_in_sync
        print("Connected to TPU Successfully:\n TPUs Initialised with Replicas:",REPLICAS)
        
        return strategy
    
    except ValueError:
        
        print("Connection to TPU Falied")
        print("Using default strategy for CPU and single GPU")
        strategy = tf.distribute.get_strategy()
        
        return strategy
    
strategy=Init_TPU()

# Data

In [ ]:
# Define Dataset Path
path = '../input/contradictory-my-dear-watson/'

In [ ]:
# Load Training Data
train_url = os.path.join(path,'train.csv')
train_data = pd.read_csv(train_url, header='infer')

# Sample Submission
sample_sub_url = os.path.join(path,'sample_submission.csv')
sample_sub = pd.read_csv(sample_sub_url, header='infer')

# Load Test Data
test_url = os.path.join(path,'test.csv')
test_data = pd.read_csv(test_url, header='infer')

# EDA

In [ ]:
print("Total Records: ", train_data.shape[0])

In [ ]:
#Inspect
train_data.head()

In [ ]:
# Records per Label
print("Records per Label: \n", train_data.groupby('label').size())

In [ ]:
# Records per Languages
print("Records per Language: \n", train_data.groupby('language').size())

In [ ]:
# Pie Chart
df = pd.DataFrame({"count": train_data.language.value_counts() })
fig = px.pie(df, values='count', names=df.index, title='Language Count %',
             labels={'index':'lang'}, color_discrete_sequence=px.colors.diverging.Earth)
fig.update_traces(textinfo='percent')
fig.show()

In [ ]:
# Bar Plot - Label Count per Language 
fig, ax = plt.subplots(figsize=(20,10))
train_data.groupby(['language','label']).count()['premise'].unstack().plot(ax=ax,kind='bar', cmap='cividis')
plt.grid(color='gray',linestyle='--',linewidth=0.2)
ax.set_facecolor('#d8dcd6')
plt.title("Label Count per Language", fontsize='18')
plt.xticks(rotation=45)

In [ ]:
# Garbage Collection
gc.collect()

# Data Prep

In this step we shall use a pre-trained Transformer model from HuggingFace to encode the input before it is fed to the model

* For details about XLM-RoBERTa model check [here](https://huggingface.co/transformers/model_doc/xlmroberta.html#)

**Model = xlm-roberta-large**


In [ ]:
# Transformer Model Name
transformer_model = 'jplu/tf-xlm-roberta-large'

# Define Tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained(transformer_model)

In [ ]:
# Checking the output of tokenizer
tokenizer.convert_tokens_to_ids(list(tokenizer.tokenize("Elementary, My Dear Watson!")))

In [ ]:
# Create seperate list from Train & Test Dataframes with only Premise & Hypothesis
train = train_data[['premise','hypothesis']].values.tolist()
test = test_data[['premise','hypothesis']].values.tolist()

In [ ]:
# Define Max Length
max_len = 80   # << change if you wish

# Encode the training & test data 
train_encode = tokenizer.batch_encode_plus(train, pad_to_max_length=True, max_length=max_len)
test_encode = tokenizer.batch_encode_plus(test, pad_to_max_length=True, max_length=max_len)

In [ ]:
# Split the Training Data into Training (90%) & Validation (10%)

test_size = 0.1  # << change if you wish
x_train, x_val, y_train, y_val = train_test_split(train_encode['input_ids'], train_data.label.values, test_size=test_size)


# Split Test Data
x_test = test_encode['input_ids']

In [ ]:
#garbage collect
gc.collect()

In [ ]:
# Loading Data Into TensorFlow Dataset
AUTO = tf.data.experimental.AUTOTUNE
batch_size = 16 * strategy.num_replicas_in_sync

train_ds = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).repeat().shuffle(2048).batch(batch_size).prefetch(AUTO))
val_ds = (tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size).prefetch(AUTO))

test_ds = (tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size))

### Prefetching

Prefetching overlaps the preprocessing and model execution of a training step. While the model is executing training step s, the input pipeline is reading the data for step s+1. Doing so reduces the step time to the maximum (as opposed to the sum) of the training and the time it takes to extract the data.

The tf.data API provides the tf.data.Dataset.prefetch transformation. It can be used to decouple the time when data is produced from the time when data is consumed. In particular, the transformation uses a background thread and an internal buffer to prefetch elements from the input dataset ahead of the time they are requested. The number of elements to prefetch should be equal to (or possibly greater than) the number of batches consumed by a single training step. You could either manually tune this value, or set it to tf.data.experimental.AUTOTUNE which will prompt the tf.data runtime to tune the value dynamically at runtime.

In [ ]:
# Garbage Collection
gc.collect()

# Build & Train Model

Now we shall build a model with the pre-trained BERT transformer model into Keras Functional Model

In [ ]:
def build_model(strategy,transformer):
    with strategy.scope():
        transformer_encoder = TFXLMRobertaModel.from_pretrained(transformer)  #Pretrained BERT Transformer Model
        
        input_layer = Input(shape=(max_len,), dtype=tf.int32, name="input_layer")
        
        sequence_output = transformer_encoder(input_layer)[0]
        
        cls_token = sequence_output[:, 0, :]
        
        output_layer = Dense(3, activation='softmax')(cls_token)
        
        model = Model(inputs=input_layer, outputs=output_layer)
        
        model.compile(
            Adam(lr=1e-5), 
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy']
        )
        
        return model
    

# Applying the build model function
model = build_model(strategy,transformer_model)

In [ ]:
# Model Summary
model.summary()

In [ ]:
# Train the Model

epochs = 30  # < change if you wish
n_steps = len(train_data) // batch_size 

model.fit(train_ds, 
          steps_per_epoch = n_steps, 
          validation_data = val_ds,
          epochs = epochs)

In [ ]:
# Garbage Collection
gc.collect()

# Prediction & Saving

In [ ]:
prediction = model.predict(test_ds, verbose=0)
sample_sub['prediction'] = prediction.argmax(axis=1)

In [ ]:
sample_sub.to_csv("submission.csv", index=False)